In [ ]:
import requests
import zipfile
import io
import pandas as pd

1. Descarga de datos
Descarga los datasets para vinos tintos y blancos desde la siguiente URL: Wine Quality Dataset.
 - Asegúrate de que ambos datasets se encuentren disponibles en tu entorno de trabajo como archivos CSV.
 - Idealmente, intenta llevarlo a cabo de manera programática para no tener que hacer operaciones manualmente.

In [ ]:
# URL  datasets - ZIP 
url = "https://archive.ics.uci.edu/static/public/186/wine+quality.zip"

# Descarga archivo ZIP 
response = requests.get(url)
z = zipfile.ZipFile(io.BytesIO(response.content))
z.extractall("wine_quality")  # Extraer zip en la carpeta wine_quality

In [ ]:
# Carga archivos csv
df_red = pd.read_csv("wine_quality/winequality-red.csv",sep=";")
df_white = pd.read_csv("wine_quality/winequality-white.csv", sep=";")

In [ ]:
df_red.head()

In [ ]:
df_white.head()

2. Combinar los datos

- Usa Pandas para cargar ambos datasets en dataframes en memoria. - Combina los datos en un único dataframe añadiendo una columna adicional que indique el tipo de vino (`red` o `white`). - ¿Cuántos registros tenemos? ¿Cuántas variables y de qué tipo?

In [ ]:
#Añadir columna type_wine
df_red['type_wine']="red"
df_white['type_wine']="white"

In [ ]:
#Combinar los dos csv
df_wine = pd.concat([df_red, df_white], ignore_index=True)

In [ ]:
df_wine

In [ ]:
num_registros = df_wine.shape[0]
num_variables = df_wine.shape[1]
tipos_variables = df_wine.dtypes

In [ ]:
print(f"Registros: {num_registros}")
print(f"Variables: {num_variables}")

In [ ]:
print(f"Tipos de variables:\n{tipos_variables}")

3. Filtrar atípicos y manejar datos ausentes

- Realiza un análisis estadístico o inspección visual de cada columna numérica para identificar valores atípicos. - Usa Pandas para filtrar y eliminar los datos atípicos y los valores ausentes. Explica en tu entrega qué criterios utilizaste para identificar los atípicos.

In [ ]:
# Descripción estadística básica
desc_stats = df_wine.describe()

In [ ]:
print("Estadísticas Descriptivas:")
print(desc_stats)

In [ ]:
#Identificar valores atípicos utilizando IQR
df_wine_numeric = df_wine.select_dtypes(include=['float64', 'int64'])
Q1 = df_wine_numeric.quantile(0.25)
Q3 = df_wine_numeric.quantile(0.75)
IQR = Q3 - Q1

In [ ]:
# Calcular los límites inferior y superior
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR
mask = (df_wine_numeric >= lower_bound) & (df_wine_numeric <= upper_bound)

In [ ]:
# Filtrar los valores atípicos
df_wine_filtered = df_wine_numeric[mask.all(axis=1)]

# Eliminar los valores ausentes
df_wine_end = df_wine_filtered.dropna()

In [ ]:
# Seleccionar las columnas de texto del DataFrame original
df_wine_text = df_wine.select_dtypes(include=['object', 'string'])

# Combinar el DataFrame filtrado numérico con las columnas de texto
df_final = pd.concat([df_wine_end, df_wine_text], axis=1)
df_final.to_csv('vinos.csv', index=False)

In [ ]:
# Ordenar el DataFrame por 'nombre_columna' de manera ascendente
df_ordenado = df_final.sort_values(by='fixed acidity', ascending=True)

# Guardar el DataFrame ordenado en un archivo CSV sin índice
df.to_csv('archivo_ordenado.csv', index=False)

In [ ]:
print(f"DataFrame después de eliminar valores atípicos: \n\n{df_final.describe()}")

4. Almacenar los datos limpios en SQLite

- Usa SQLite para almacenar el dataframe limpio en una base de datos persistente. - Sigue la documentación oficial de SQLite: [SQLite Python Documentation](https://docs.python.org/3/library/sqlite3.html).

In [ ]:
import sqlite3

# Creamos la base de datos SQLite
conn = sqlite3.connect('wines_sqlite.db')

# Guardar el dataframe en una tabla llamada 'vinos'
df_final.to_sql('wines', conn, if_exists='replace', index=False)

print("Datos guardados en la base de datos SQLite.")

# Cerrar la conexión
conn.close()

5. Realizar 3 consultas en SQLite

Basándote en los datos y las columnas del dataset, realiza las siguientes consultas:


In [ ]:
# Reconectamos con la base de datos
conn = sqlite3.connect('wines_sqlite.db')


1. Consulta 1: ¿Cuál es el promedio de calidad (`quality`) por tipo de vino (`type`)?

In [ ]:
# Consulta 1: Promedio de calidad por tipo
query1 = """
SELECT type_wine, AVG(quality) as avg_quality
FROM wines
GROUP BY type_wine
"""
df_query1 = pd.read_sql_query(query1, conn)
print("Consulta 1 - Promedio de Calidad por Tipo:")
print(df_query1)

2. Consulta 2: ¿Cuántos vinos tienen un nivel de alcohol superior a 10.5, agrupados por tipo?

In [ ]:
# Consulta 2: Conteo de vinos con alcohol > 10.5 agrupados por tipo
query2 = """
SELECT type_wine, COUNT(*) as count_alcohol_high
FROM wines
WHERE alcohol > 10.5
GROUP BY type_wine
"""
df_query2 = pd.read_sql_query(query2, conn)
print("\nConsulta 2 - Vinos con Alcohol > 10.5:")
print(df_query2)

3. Consulta 3: Obtén el conteo de vinos por nivel de acidez (`fixed acidity`) agrupados en rangos (por ejemplo, de 0-5, 5-10, 10-15).

In [ ]:
# Consulta 3: Conteo de vinos por rango de acidez
query3 = """
WITH ranges AS (
    SELECT '0-5' AS acidity_range
    UNION ALL
    SELECT '5-10'
    UNION ALL
    SELECT '10-15'
    UNION ALL
    SELECT '15+'
)
SELECT 
    r.acidity_range,
    COALESCE(w.count, 0) AS count
FROM ranges r
LEFT JOIN (
    SELECT 
        CASE 
            WHEN [fixed acidity] < 5 THEN '0-5'
            WHEN [fixed acidity] >= 5 AND [fixed acidity] < 10 THEN '5-10'
            WHEN [fixed acidity] >= 10 AND [fixed acidity] < 15 THEN '10-15'
            ELSE '15+'
        END AS acidity_range,
        COUNT(*) AS count
    FROM wines
    GROUP BY acidity_range
) w ON r.acidity_range = w.acidity_range
ORDER BY 
    CASE 
        WHEN r.acidity_range = '15+' THEN 1
        WHEN r.acidity_range = '10-15' THEN 2
        WHEN r.acidity_range = '5-10' THEN 3
        WHEN r.acidity_range = '0-5' THEN 4
    END
"""
df_query3 = pd.read_sql_query(query3, conn)
print("\nConsulta 3 - Conteo por Rango de Acidez:")
print(df_query3)


In [ ]:
# Cerramos la conexión con la base de datos. 
conn.close()

6. Exportar datos a JSONLines

De cara a una potencial insercion en una base de datos noSQL como `mongoDB`, podemos servirnos de pandas para preparar los datos.

- ¿Qué estructura de datos de python es la más similar a un documento noSQL?



- Usa Pandas para transformar los datos de una de las consultas en un archivo JSONLines.

- Usa la librería `jsonlines` para guardar el archivo.


- ¿Qué problemas podrían surgir al transformar un dataframe en jsonlines?

- Añade una columna que sea originalmente un `np.array`,¿qué sucede al transformarlo en jsonlines?

- Añade una columna que sea originalmente un `pd.datetime`,¿qué sucede al transformarlo en jsonlines?

7. Análisis de calidad de los vinos

- Inspecciona qué caracteriza a los vinos tintos y blancos con mayor calidad (`quality`).

- Usa análisis estadístico, gráficos o cualquier técnica que consideres relevante para identificar patrones.